In [277]:
import pandas as pd
import numpy as np

'''
df_full_ipo_date_97_to_19_silicon_valley - This is the data that I scraped from the NASDAQ website that lists all the
IPO's in Silicon Valley betnween 1997 and 2019/04. I will need to add updated IPO results if any new IPO's are announced
but for right now this is static --> REAL TIME WAY TO UPDATE WHEN NEW IPOS ARE ANNOUNCED
zip_all_census_data_silicon_valley - census data from all cities and beaureaus -- that holds econometric stats but also 
race, sex, and age stats about each zipcode
df_all_home_by_zip_silicon_valley - All home prices from 1996 to 2019
'''


In [584]:
'''
data processing to only view california real estate data

df_median_all_home_price_by_zip = pd.read_csv('data/raw/Zip_MedianListingPrice_AllHomes.csv',  encoding="ISO-8859-1")
df_single_family_residence_by_zip = pd.read_csv('data/raw/Zip_Zhvi_SingleFamilyResidence.csv',  encoding="ISO-8859-1")

df_median_all_home_price_by_zip_california = df_median_all_home_price_by_zip[df_median_all_home_price_by_zip['State']=='CA']
df_single_family_residence_by_zip_california = df_single_family_residence_by_zip.loc[df_single_family_residence_by_zip['State']=='CA']
'''
df_sale_counts_by_zip = pd.read_csv('data/raw/Sale_Counts_Zip.csv', encoding="ISO-8859-1")
df_all_home_by_zip = pd.read_csv('data/raw/Zip_Zhvi_AllHomes.csv',  encoding="ISO-8859-1")

df_sale_counts_by_zip_california = df_sale_counts_by_zip.loc[df_sale_counts_by_zip['StateName']=='California']
df_all_home_by_zip_california = df_all_home_by_zip.loc[df_all_home_by_zip['State']=='CA']


In [585]:
# data processing to load all IPO Data between 1997 and present data. This data has been scraped using code from src/web_scrapers.py
df_full_ipo_date_97_to_19 = pd.read_csv('data/processed/1997-04_2019_full_ipo_data.csv',encoding="ISO-8859-1")

df_ritter_ipo_data_97_to_19 = pd.read_csv('data/raw/ipo_ritter_data.csv',encoding="ISO-8859-1")


In [586]:
#data processing of census data to create demographics by zip code
zip_census_data = pd.read_csv('data/raw/zip_census_bureau_economic_characteristics_2017.csv',encoding="ISO-8859-1")
zip_census_age_race_data = pd.read_csv('data/raw/zip_census_bureau_age_race_2017.csv',encoding="ISO-8859-1")

In [587]:
# type wrangling of all pandas series in each dataframe
df_sale_counts_by_zip_california["RegionID"] = df_sale_counts_by_zip_california["RegionID"].astype(dtype=np.int64)
df_sale_counts_by_zip_california['StateName'] = df_sale_counts_by_zip_california['StateName'].astype('str')

df_full_ipo_date_97_to_19['Date Filed'] =  pd.to_datetime(df_full_ipo_date_97_to_19['Date Filed'], errors='coerce', format='%m/%d/%Y')
df_full_ipo_date_97_to_19['Lockup Expiration Date'] =  pd.to_datetime(df_full_ipo_date_97_to_19['Lockup Expiration Date'], errors='coerce', format='%m/%d/%Y')



/Users/aaron/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/aaron/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [588]:
# data wrangling of census data

zip_census_data.rename(columns = {'Id2':'Zipcode'}, inplace=True)
zip_census_data.rename(columns = {'Percent; EMPLOYMENT STATUS - Civilian labor force - Unemployment Rate':'Unemployment Rate'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Less than $10,000':'l10000'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $10,000 to $14,999':'l15000'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999':'l25000'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)':'Mean Travel Time to Work Estimate (minutes)'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017| INFLATION-ADJUSTED DOLLARS) - Total households - $200,000 or more':'Percent of Households with Income Greater than $200,000'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars)':'Median Household Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars)':'Mean Household Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)':'Per Capita Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage':'Percent of Population with no Health Insurance Coverage'}, inplace=True)
zip_census_data.rename(columns = {'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people':'Percent of People whose Income in the Past 12 months has been Below Poverty Level'}, inplace=True)


filtered_census_data_columns = ['Zipcode',
                               'Unemployment Rate',
                               'Mean Travel Time to Work Estimate (minutes)',
                               'Percent of Households with Income Greater than $200,000',
                               'Median Household Income Estimate (dollars)',
                               'Mean Household Income Estimate (dollars)',
                               'Per Capita Income Estimate (dollars)',
                               'Percent of Population with no Health Insurance Coverage',
                               'Percent of People whose Income in the Past 12 months has been Below Poverty Level',
                               'Percent of Households With Income Less Than $24,999']
zip_census_data['l10000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l10000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l10000'] = zip_census_data['l10000'].astype(float)

zip_census_data['l15000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l15000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l15000'] = zip_census_data['l15000'].astype(float)

zip_census_data['l25000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l25000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l25000'] = zip_census_data['l25000'].astype(float)
zip_census_data["Percent of Households With Income Less Than $24,999"] = zip_census_data['l10000'] + zip_census_data['l15000']+zip_census_data['l25000']
zip_census_data = zip_census_data.filter(filtered_census_data_columns)

zip_census_age_race_data.rename(columns = {'Id2':'Zipcode'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Estimate; SEX AND AGE - Median age (years)':'Median Age'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - Under 18 years':'Percent of People under 18 years of age'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - 65 years and over':'Percent of People 65 years and over'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - 18 years and over - Male':'Percent of Males'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - 18 years and over - Female':'Percent of Females'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race)':'Percent of People who are Hispanic'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - White alone':'Percent of People who are White'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Black or African American alone':'Percent of People who are Black or African American'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Asian alone':'Percent of People who are Asian'}, inplace=True)

filtered_census_age_race_data_columns = ['Zipcode',
                               'Median Age',
                               'Percent of People under 18 years of age',
                               'Percent of People 65 years and over',
                               'Percent of Males',
                               'Percent of Females',
                               'Percent of People who are Hispanic',
                               'Percent of People who are White',
                               'Percent of People who are Black or African American',
                               'Percent of People who are Asian']
zip_census_age_race_data = zip_census_age_race_data.filter(filtered_census_age_race_data_columns)


In [466]:
'''
***DONT RUN THE NEXT 3 IF THESE ARE ALREADY CREATED***
creating list and dictionary for storing all the zipcodes in silicon valley + san Francisco
'''
import csv
zipcodes = {} # structured with within 5, 10 miles from another zipcode
zip_list=[]

def add_zipcodes(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        your_list = list(reader)
        for z in range(0, len(your_list)):
            zipcodes[your_list[z][0]]= [],[]
            zip_list.append(your_list[z][0])

with open('data/raw/Santa_Clara_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])
    
with open('data/raw/San_Mateo_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])
    
with open('data/raw/San_Francisco_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])
    
with open('data/raw/Alameda_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])



len(zip_list)

173

In [467]:
import urllib.request
import json

def calculate_distance_between_zips(zipcode,min_radius, max_radius):
    # api-endpoint 
    URL_base = "https://api.zip-codes.com/ZipCodesAPI.svc/1.0/FindZipCodesInRadius?zipcode="
  
    URL = URL_base+zipcode+'&minimumradius='+min_radius+'&maximumradius='+max_radius+'&key=DEMOAPIKEY'
    # sending get request and saving the response as response object 
    contents = urllib.request.urlopen(URL).read()
  

    # printing the output 
    zipcodes_nearby =[]
    for i in range(1, len(json.loads(contents)['DataList'])):
        zipcodes_nearby.append(json.loads(contents)['DataList'][i]['Code'])
        
    return zipcodes_nearby

In [468]:
'''
***DONT RUN IF THESE ARE ALREADY CREATED***
Create zipcodes distance list
'''
for i in range(0, len(zip_list)):
    print(zip_list[i])
    zipcodes[zip_list[i]]= calculate_distance_between_zips(zip_list[i], '0','5'), calculate_distance_between_zips(zip_list[i], '5','10')
    

95002
95008
95013
95014
95020
95023
94550
94022
94024
95030
95032
95033
95035
95037
95140
94040
94041
94043
94301
94303
94304
94306
94028
95110
95111
95112
95113
95116
95117
95118
95119
95120
95121
95122
95123
95124
95125
95126
95127
95128
95129
95130
95131
95132
95133
95134
95135
95136
95138
95139
95148
95046
95050
95051
95054
95053
95070
94305
94085
94086
94087
94089
95076
94027
94002
94005
94010
94014
94015
94019
94020
94021
94025
94030
94037
94038
94044
94303
94060
94028
94061
94062
94063
94065
94066
94070
94128
94074
94401
94402
94403
94404
94080
94102
94103
94104
94105
94107
94108
94109
94110
94111
94112
94114
94115
94116
94117
94118
94121
94122
94123
94124
94127
94129
94130
94131
94132
94133
94134
94158
94501
94502
94706
94702
94703
94704
94705
94707
94708
94709
94710
94720
94514
94546
94552
94505
94568
94608
94536
94538
94539
94555
94541
94542
94544
94545
94550
94551
94560
94601
94602
94603
94605
94606
94607
94609
94610
94611
94612
94618
94619
94621
94613
94566
94588
94577
9457

In [589]:
from datetime import datetime  
from datetime import timedelta  

'''
processing to merge all zip code data into one dataframe to create a feature
from 
1) df_all_home_by_zip_silicon_valley
2) zip_all_census_data_silicon_valley
'''
 
zip_census_age_race_data_silicon_valley = zip_census_age_race_data[zip_census_age_race_data['Zipcode'].isin(zip_list)]
zip_census_data_silicon_valley = zip_census_data[zip_census_data['Zipcode'].isin(zip_list)]
df_sale_counts_by_zip_silicon_valley = df_sale_counts_by_zip_california[df_sale_counts_by_zip_california['RegionName'].isin(zip_list)]
# the one we should use
df_all_home_by_zip_silicon_valley = df_all_home_by_zip_california[df_all_home_by_zip_california['RegionName'].isin(zip_list)]
# df_single_family_residence_by_zip_silicon_valley only includes detached homes, so super expensive
df_single_family_residence_by_zip_silicon_valley = df_single_family_residence_by_zip_california[df_single_family_residence_by_zip_california['RegionName'].isin(zip_list)]
df_median_all_home_price_by_zip_silicon_valley = df_median_all_home_price_by_zip_california[df_median_all_home_price_by_zip_california['RegionName'].isin(zip_list)]
df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19[df_full_ipo_date_97_to_19['Zipcode'].isin(zip_list)]

df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19_silicon_valley.drop(['Lockup Expiration Date', 'Lockup Period'], axis=1)

df_all_home_by_zip_silicon_valley = df_all_home_by_zip_silicon_valley.drop(['City', 'RegionID','State','Metro','CountyName', 'SizeRank'], axis=1)
df_sale_counts_by_zip_silicon_valley = df_sale_counts_by_zip_silicon_valley.drop(['RegionID','StateName','SizeRank', 'seasAdj'], axis=1)

# causing warning -- issues with grabbing the 180, but all lockup periods here are 180
df_full_ipo_date_97_to_19_silicon_valley['Lockup Expiration Date'] = df_full_ipo_date_97_to_19_silicon_valley['Date Filed']+ timedelta(days=180)


In [598]:
export_csv = df_zip_all_joined_data.to_csv ('data/processed/df_zip_all_joined_data.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = df_all_home_by_zip_silicon_valley.to_csv ('data/processed/df_all_home_by_zip_silicon_valley.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = zip_census_data_silicon_valley.to_csv ('data/processed/export_dataframe.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = df_full_ipo_date_97_to_19_silicon_valley.to_csv ('data/processed/df_full_ipo_date_97_to_19_silicon_valley.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path

with open('data/processed/zipcodes_within_radius.txt', 'w') as json_file:
  json.dump(zipcodes, json_file)


In [602]:
'''
*** DONT RUN MULTIPLE TIMES. ALL HOMES AND SALES COUNTS PREFIX WILL BE ADDED MULTIPLE TIMES***
1) join census bureau dataframes on -- only joining now to minimize data to silicon valley zipcodes that we will use.
key is zip code
2) join with df_all_home_by_zip_silicon_valley
3) join with df_sale_counts_by_zip_silicon_valley
'''
df_all_home_by_zip_silicon_valley.columns = df_all_home_by_zip_silicon_valley.columns.str.replace('All Homes ', '')
df_sale_counts_by_zip_silicon_valley.columns = df_sale_counts_by_zip_silicon_valley.columns.str.replace('Sales Counts ', '')

df_all_home_by_zip_silicon_valley = df_all_home_by_zip_silicon_valley.add_prefix('All Homes ')
df_sale_counts_by_zip_silicon_valley = df_sale_counts_by_zip_silicon_valley.add_prefix('Sales Counts ')
zip_all_census_data_silicon_valley = zip_census_age_race_data_silicon_valley.set_index('Zipcode').join(zip_census_data_silicon_valley.set_index('Zipcode'))

df_all_home_by_zip_silicon_valley.rename(columns = {'All Homes RegionName':'Zipcode'}, inplace=True)
df_sale_counts_by_zip_silicon_valley.rename(columns = {'Sales Counts RegionName':'Zipcode'}, inplace=True)




In [591]:
#df_zip_all_joined_data = zip_all_census_data_silicon_valley.set_index('Zipcode').join(df_all_home_by_zip_silicon_valley.set_index('Zipcode'))
#df_zip_all_joined_data = df_zip_all_joined_data.set_index('Zipcode').join(df_sale_counts_by_zip_silicon_valley.set_index('Zipcode'))

df_zip_all_joined_data = pd.merge(zip_all_census_data_silicon_valley, df_all_home_by_zip_silicon_valley, on='Zipcode', how='inner')
df_zip_all_joined_data = pd.merge(df_zip_all_joined_data, df_sale_counts_by_zip_silicon_valley, on='Zipcode', how='inner')


In [592]:
df_ritter_ipo_data_97_to_19

,IPO Name,Offer date,Ticker,CUSIP,PERM,Found
0,ROYSTER,19750130.0,NaN,78088610,67898,1901
1,VARCO,19750609.0,VRC,92212610,63044,1908
2,COORS ADOLPH,19750610.0,TAP,21701610,59248,1901
3,KEYSTONE FOODS CORP,19750715.0,NaN,49348410,61989,1965
4,C. F. BRAUN,19750826.0,NaN,10564710,58579,1909
5,BROOKS FASHION STORES,19750910.0,NaN,11439410,65517,1923
6,"CHRISTENSEN, INC.",19750924.0,NaN,17056510,23342,1944
7,KERR GLASS MANUFACTURI,19751021.0,KGM,49237610,58616,1903
8,LIQUID AIR,19751028.0,LANA,53631510,49825,1965
9,OCEANEERING INTERNATL,19751030.0,OII,67523210,58975,1965


In [594]:
'''
wrangling ritter data
'''
df_ritter_ipo_data_97_to_19.rename(columns = {'Ticker':'Symbol'}, inplace=True)

df_ritter_ipo_data_97_to_19["Found"] = df_ritter_ipo_data_97_to_19["Found"].astype(dtype=np.int64)





In [595]:
df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19_silicon_valley.set_index('Symbol').join(df_ritter_ipo_data_97_to_19.set_index('Symbol'), how='left')


In [596]:
df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19_silicon_valley.drop(['IPO Name', 'Offer date','CUSIP','PERM'], axis=1)


In [597]:
df_full_ipo_date_97_to_19_silicon_valley


,Company Name,Offer Amount,Date Filed,Zipcode,Number of Employees,CIK,Lockup Expiration Date,Found
Symbol,,,,,,,,
A,AGILENT TECHNOLOGIES INC,2.160000e+09,1999-08-16,95051,45800,1090872,2000-02-12,1949
AATI,ADVANCED ANALOGIC TECHNOLOGIES INC,1.060000e+08,2005-04-04,95054,183,1104042,2005-10-01,1969
AATI,ADVANCED ANALOGIC TECHNOLOGIES INC,1.060000e+08,2005-04-04,95054,183,1104042,2005-10-01,1997
AAVL,"ADVERUM BIOTECHNOLOGIES, INC.",1.020000e+08,2014-06-30,94025,14,1501756,2014-12-27,2006
ABGX,ABGENIX INC,2.000000e+07,1998-04-03,94555,57,1052837,1998-09-30,1996
ABOV,ABOVENET COMMUNICATIONS INC,6.500000e+07,1998-09-10,95113,78,1026819,1999-03-09,1996
ACCL,ACCELGRAPHICS INC,2.340000e+07,1997-02-07,95112,55,947798,1997-08-06,1994
ACCL,ACCELGRAPHICS INC,2.340000e+07,1997-02-07,95112,55,947798,1997-08-06,1996
ACEI,PACIFIC SYSTEMS CONTROL TECHNOLOGY INC,6.500000e+06,1997-03-21,94065,38,1034840,1997-09-17,1997


In [603]:
df_all_home_by_zip_silicon_valley

,All Homes Zipcode,All Homes 1996-04,All Homes 1996-05,All Homes 1996-06,All Homes 1996-07,All Homes 1996-08,All Homes 1996-09,All Homes 1996-10,All Homes 1996-11,All Homes 1996-12,...,All Homes 2018-07,All Homes 2018-08,All Homes 2018-09,All Homes 2018-10,All Homes 2018-11,All Homes 2018-12,All Homes 2019-01,All Homes 2019-02,All Homes 2019-03,All Homes 2019-04
12,94109,300300.0,300300.0,300300.0,300300.0,300400.0,300600.0,301000.0,301600.0,302600.0,...,1233600,1234400,1242400,1257900,1265300,1263400,1261200,1253300,1237500,1224100
79,94501,213300.0,212900.0,212500.0,212200.0,212000.0,212100.0,212500.0,213200.0,214300.0,...,981500,973000,970300,977000,984100,985300,982100,978100,977500,976800
87,94110,215700.0,216400.0,217400.0,218600.0,220100.0,221800.0,223900.0,226400.0,229500.0,...,1543700,1549400,1552200,1551500,1545800,1541700,1537800,1527300,1515200,1508900
210,94536,215000.0,215400.0,216000.0,216800.0,217700.0,218700.0,219900.0,221200.0,222700.0,...,1061000,1067900,1079900,1091000,1093700,1091800,1088100,1082000,1072500,1061100
252,95123,215900.0,216400.0,217000.0,217700.0,218600.0,219500.0,220800.0,222200.0,224000.0,...,1004600,1005300,1010800,1011400,1004700,996000,987600,974600,959000,945000
293,94544,161900.0,161900.0,162000.0,162100.0,162200.0,162400.0,162600.0,162800.0,163200.0,...,646100,649600,656500,661300,662200,661500,658700,654900,650500,645500
312,95051,241600.0,242600.0,243600.0,244700.0,245800.0,247000.0,248500.0,250100.0,252300.0,...,1505900,1513800,1529900,1531500,1512200,1490300,1468700,1439200,1405500,1381000
328,94568,247200.0,249000.0,251100.0,253300.0,255200.0,257000.0,258500.0,259700.0,261100.0,...,909600,913900,921500,926200,926100,924200,920600,915200,907300,897800
343,95035,222700.0,222900.0,223200.0,223600.0,224200.0,224900.0,225900.0,227200.0,228900.0,...,1114800,1116800,1125000,1127100,1119800,1111400,1103900,1089000,1068300,1051100
363,94122,274200.0,275100.0,276200.0,277400.0,278800.0,280300.0,282100.0,284200.0,286700.0,...,1425200,1434000,1444600,1453700,1452100,1448600,1449100,1445000,1438600,1435900
